In [1]:
import numpy as np
import tensorflow.keras as keras
import librosa
import os
import math
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def predict(model, X, y):

    
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    
    prediction = model.predict(X)

    
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))


In [3]:
from keras.models import load_model

In [4]:
# Load models
cnn_model = load_model("/kaggle/input/kaggle-working-saved/cnn_audio_fin.h5")
lstm_model = load_model("/kaggle/input/kaggle-working-saved/lstm_audio.h5")

In [5]:
def predict_voice(audio_file_path, genre_mapping):

    # Load audio file
    signal, sample_rate = librosa.load(audio_file_path, sr=22050)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=13, n_fft=2048, hop_length=512)
    mfcc = mfcc.T

    # Resize MFCC to the appropriate dimension
    mfcc = np.resize(mfcc, (130, 13, 1))

    # Reshape MFCC to the appropriate dimension
    mfcc = mfcc[np.newaxis, ...]

    # Make prediction using the model
    cnn_prediction = cnn_model.predict(mfcc)
    lstm_prediction = lstm_model.predict(mfcc)
    prediction = (cnn_prediction + lstm_prediction) / 2.0
    predicted_index = np.argmax(prediction, axis=1)

    # Map predicted index to genre label
    genre_label = genre_mapping[predicted_index[0]]
    print("Raw prediction:", prediction)

    return genre_label


In [8]:
# enter your audio file path here
audio_file_path = ''
genre_mapping = {1: "spoof", 0: "bonafide"}


predicted_voice = predict_voice(audio_file_path, genre_mapping)

print("Predicted label:", predicted_voice)

1/1 [==============================] - 0s 21ms/step
Raw prediction: [[0.00469366 0.9953064 ]]
Predicted label: spoof


In [ ]:
# folder_dir = '/kaggle/input/spoof-la-dataset'
# test_(folder_dir)